In [1]:
from pathlib import Path
import ast
import duckdb
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

rootpath = Path.cwd().parent
duckdb_destination_path = rootpath / "temp/curve.duckdb"


In [2]:
sql = """
SELECT * FROM information_schema.tables
"""
conn = duckdb.connect(str(duckdb_destination_path))
df = conn.execute(sql).fetchdf()

display(df)
conn.close()


,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,curve,controllers,logs,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,curve,controllers,_dlt_loads,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,curve,controllers,_dlt_pipeline_state,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,curve,controllers,_dlt_version,BASE TABLE,None,None,None,None,None,YES,NO,None,None


## 

## Borrower Concentration

### Transformations


In [3]:
sql = """
SELECT * FROM curve.controllers.logs
"""
conn = duckdb.connect(str(duckdb_destination_path))
df_log = conn.execute(sql).fetchdf()#.drop(columns=['_dlt_load_id', '_dlt_id'])
df_log.head(3)

,topics,address,data,block_number,block_hash,time_stamp,gas_price,gas_used,log_index,transaction_hash,transaction_index,_dlt_load_id,_dlt_id
0,"[""0xeec6b7095a637e006c79c1819d696e353a8f703db2...",0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x1075dc7,0x6f971a452a974a94c9d59bf20c1c6ed66bb5c0ff2782...,0x64612d33,0x9c2847d19,0x1287b2,0xa6,0xcac35a9c033ea94fa885060c2fcce68ac17574250cc6...,0x3f,1750953848.3246627,IGPaA++ijLN4tQ
1,"[""0xe1979fe4c35e0cef342fef5668e2c8e7a7e9f5d5d1...",0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x1075dc7,0x6f971a452a974a94c9d59bf20c1c6ed66bb5c0ff2782...,0x64612d33,0x9c2847d19,0x1287b2,0xa7,0xcac35a9c033ea94fa885060c2fcce68ac17574250cc6...,0x3f,1750953848.3246627,yK0KInKh8Z2nhw
2,"[""0xeec6b7095a637e006c79c1819d696e353a8f703db2...",0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x1075f25,0x0c72e1aa14e2607d062d6acc56e088d299cb9aedea2a...,0x64613def,0xc0ff737d8,0x141d07,0x62,0xf5cc3b8d70eb97313916d62acdb7936b4996d8c8d430...,0x3e,1750953848.3246627,gU+qZfLPkrmyew


In [4]:
topic_signature = '0xeec6b7095a637e006c79c1819d696e353a8f703db2c49fc0219e17a8fd04f7f2'

# transform topics to list
df_log['topics'] = df_log['topics'].apply(ast.literal_eval)

# filter by topic signature
df = df_log[df_log['topics'].str[0] == topic_signature]
df.reset_index(drop=True, inplace=True)

# extract user address
df['user'] = df['topics'].str[1].apply(lambda x: "0x" + x[26:])

# transform time_stamp to datetime
df['time_stamp'] = pd.to_datetime(df['time_stamp'].apply(lambda x: int(x, 16)), unit='s')

In [5]:
def decode_user_state_data(data: str) -> dict:
    """
    Decode Solidity function calldata string with arguments: uint256 collateral, uint256 debt, int256 n1, int256 n2, uint256 liquidation_discount
    
    Args:
        calldata_str: string containing hex calldata with '0x' prefix
        
    Returns:
        dict with decoded values: collateral, debt, n1, n2, liquidation_discount
    """
    def hex_to_int256(hex_str):
        """Convert hex string to int256, handling negative values"""
        value = int(hex_str, 16)
        return value if value < 2**255 else value - 2**256
    
    def hex_to_uint256(hex_str):
        """Convert hex string to uint256"""
        return int(hex_str, 16)
    
    
    # Extract each 32-byte argument (64 hex characters each)
    result = {  
        'collateral': hex_to_uint256(data[2:66]),
        'debt': hex_to_uint256(data[66:130]),
        'n1': hex_to_int256(data[130:194]),
        'n2': hex_to_int256(data[194:258]),
        'liquidation_discount': hex_to_uint256(data[258:])
    }
    
    return result

decoded_results = df['data'].apply(decode_user_state_data)
decoded_df = pd.DataFrame(decoded_results.tolist())

df = pd.concat([df, decoded_df], axis=1)
df.head(3)

,topics,address,data,block_number,block_hash,time_stamp,gas_price,gas_used,log_index,transaction_hash,transaction_index,_dlt_load_id,_dlt_id,user,collateral,debt,n1,n2,liquidation_discount
0,[0xeec6b7095a637e006c79c1819d696e353a8f703db2c...,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x1075dc7,0x6f971a452a974a94c9d59bf20c1c6ed66bb5c0ff2782...,2023-05-14 18:49:23,0x9c2847d19,0x1287b2,0xa6,0xcac35a9c033ea94fa885060c2fcce68ac17574250cc6...,0x3f,1750953848.3246627,IGPaA++ijLN4tQ,0x7a16ff8270133f063aab6c9977183d9e72835428,929933909709140155529,1000000000000000000000000,40,49,60000000000000000
1,[0xeec6b7095a637e006c79c1819d696e353a8f703db2c...,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x1075f25,0x0c72e1aa14e2607d062d6acc56e088d299cb9aedea2a...,2023-05-14 20:00:47,0xc0ff737d8,0x141d07,0x62,0xf5cc3b8d70eb97313916d62acdb7936b4996d8c8d430...,0x3e,1750953848.3246627,gU+qZfLPkrmyew,0xd0c096ac82eba8d7a26f96ffc34b4e3bba3a1122,4950000000000000000,5000000000000000000000,44,58,60000000000000000
2,[0xeec6b7095a637e006c79c1819d696e353a8f703db2c...,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x1076eb1,0xc553074b5408478e26f408da8820402f6a01b9072097...,2023-05-15 09:30:47,0xbd28e4657,0x11201d,0x158,0x364382c084cfb2c557809367a123de63cf794f144fd5...,0x73,1750953848.3246627,s1uyLeQx3zYHCA,0x7a16ff8270133f063aab6c9977183d9e72835428,929933909709140155529,1522177275477716067202304,-1,8,60000000000000000


In [6]:
cols = [
    'address', 'time_stamp', 'user', 'collateral', 'debt', 
    # 'n1', 'n2', 'liquidation_discount', 'transaction_hash'
]
df = df[cols]
df.to_csv(rootpath / 'data/curve/borrower_state.csv', index=False)
df.head(3)

,address,time_stamp,user,collateral,debt
0,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,2023-05-14 18:49:23,0x7a16ff8270133f063aab6c9977183d9e72835428,929933909709140155529,1000000000000000000000000
1,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,2023-05-14 20:00:47,0xd0c096ac82eba8d7a26f96ffc34b4e3bba3a1122,4950000000000000000,5000000000000000000000
2,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,2023-05-15 09:30:47,0x7a16ff8270133f063aab6c9977183d9e72835428,929933909709140155529,1522177275477716067202304


### Borrower Concentration